# TensorBoard

After Training the model, run 

    tensorboard --logdir=path/to/log-directory
    
Tensorboard provides a good visualization tool for all the variables you like and works on a browser.

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def init_weights(shape, name):
    return tf.Variable(tf.random_normal(shape, stddev=0.01), name=name)

# This network is the same as the previous one except with an extra hidden layer + dropout
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden):
    # Add layer name scopes for better graph visualization
    with tf.name_scope("layer1"):
        X = tf.nn.dropout(X, p_keep_input)
        h = tf.nn.relu(tf.matmul(X, w_h))
    with tf.name_scope("layer2"):
        h = tf.nn.dropout(h, p_keep_hidden)
        h2 = tf.nn.relu(tf.matmul(h, w_h2))
    with tf.name_scope("layer3"):
        h2 = tf.nn.dropout(h2, p_keep_hidden)
        return tf.matmul(h2, w_o)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
X = tf.placeholder("float", [None, 784], name="X")
Y = tf.placeholder("float", [None, 10], name="Y")

w_h = init_weights([784, 625], "w_h")
w_h2 = init_weights([625, 625], "w_h2")
w_o = init_weights([625, 10], "w_o")

# Add histogram summaries for weights
tf.histogram_summary("w_h_summ", w_h)
tf.histogram_summary("w_h2_summ", w_h2)
tf.histogram_summary("w_o_summ", w_o)

p_keep_input = tf.placeholder("float", name="p_keep_input")
p_keep_hidden = tf.placeholder("float", name="p_keep_hidden")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
    train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    # Add scalar summary for cost
    tf.scalar_summary("cost", cost)

with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(py_x, 1)) # Count correct predictions
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float")) # Cast boolean to float to average
    # Add scalar summary for accuracy
    tf.scalar_summary("accuracy", acc_op)

In [4]:
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()

    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})
        summary, acc = sess.run([merged, acc_op], feed_dict={X: teX, Y: teY,
                                          p_keep_input: 1.0, p_keep_hidden: 1.0})
        writer.add_summary(summary, i)  # Write summary
        print(i, acc)                   # Report the accuracy

0 0.9337
1 0.9614
2 0.9706
3 0.9766
4 0.9769
5 0.9759
6 0.9786
7 0.9781
8 0.9791
9 0.9787
10 0.982
11 0.979
12 0.9815
13 0.9813
14 0.9816
15 0.9807
16 0.9811
17 0.9813
18 0.98
19 0.9823
20 0.9811
21 0.9825
22 0.9828
23 0.9833
24 0.9822
25 0.983
26 0.9831
27 0.983
28 0.982
29 0.9826
30 0.9811
31 0.9836
32 0.9843
33 0.9833
34 0.9827
35 0.9829
36 0.9836
37 0.9824
38 0.9828
39 0.9826
40 0.9831
41 0.9828
42 0.9844
43 0.9834
44 0.984
45 0.9834
46 0.9825
47 0.9831
48 0.9838
49 0.9825
50 0.9845
51 0.9839
52 0.9836
53 0.984
54 0.9837
55 0.9832
56 0.9845
57 0.983
58 0.9833
59 0.9842
60 0.9835
61 0.9841
62 0.9836
63 0.9843
64 0.9836
65 0.983
66 0.9834
67 0.9834
68 0.9833
69 0.984
70 0.9851
71 0.9824
72 0.9842
73 0.9828
74 0.9843
75 0.9853
76 0.9848
77 0.9845
78 0.9833
79 0.9837
80 0.9854
81 0.9838
82 0.9853
83 0.9859
84 0.9851
85 0.9843
86 0.9844
87 0.9841
88 0.9847
89 0.9847
90 0.9846
91 0.9837
92 0.9848
93 0.9839
94 0.9839
95 0.9846
96 0.9852
97 0.9857
98 0.9856
99 0.9854
